In [ ]:
import copy

from types import SimpleNamespace

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

from tqdm import tqdm

# Problem 1

In [ ]:
np.random.seed(0)

In [ ]:
# Constants
T = 50
C = 1
Q = 1
R = 1
theta = 1

In [ ]:
# State space model

def step_x(x, **kwargs):
    return np.cos(kwargs["theta"] * x)

def step_y(x, C, **kwargs):
    return C * x


# # State space model

# def step_x(x, Q, **kwargs):
#     return kwargs["theta"] * x + np.random.normal(0, np.sqrt(Q), size=x.shape)

# def step_y(x, C, R, **kwargs):
#     return C * x + np.random.normal(0, np.sqrt(R), size=x.shape)

In [ ]:
# Simulate data

def simulate_ssm(T, theta):
    x = np.zeros(T + 1)
    y = np.zeros(T + 1)
    x[0] = np.random.normal(0, 1)
    y[0] = x[0] + np.random.normal(0, 1)
    for t in range(1, T + 1):
        x[t] = step_x(x[t-1], theta=theta) + np.random.normal(0, np.sqrt(Q), size=x[t].shape)
        y[t] = step_y(x[t], C, theta=theta) + np.random.normal(0, np.sqrt(R), size=x[t].shape)
    return x[1:], y[1:]

In [ ]:
x_data, y_data = simulate_ssm(T, theta)
x_data.shape

In [ ]:
plt.plot(x_data)
plt.ylabel("$x_t$")
plt.xlabel("$t$");

In [ ]:
plt.plot(x_data)
plt.plot(y_data)

In [ ]:
# Fully Adapted Particle Filter (to estimate the likelihood z_hat = p(y_data|theta))

def log_op_exp(array, op=np.mean, axis=-1):
    """Uses the LogSumExp (LSE) as an approximation for the sum in a log-domain.

    :param array: Tensor to compute LSE over
    :param axis: dimension to perform operation over
    :param op: reductive operation to be applied, e.g. np.sum or np.mean
    :return: LSE
    """
    maximum = np.max(array, axis=axis)
    return np.log(op(np.exp(array - maximum), axis=axis) + 1e-8) + maximum


def fully_adapted_pf(initial_particles, step_x, C, Q, R, seed=0, verbose=True, **step_kwargs):
    """Fully adapted particle filter for a nonlinear Gaussian State Space Model.

    The importance weights are uniform for this PF.
    """
    if seed is not None:
        np.random.seed(seed)

    N = len(initial_particles)
    if verbose:
        print(f"Running with {N} particles and {step_kwargs=}")
    particles = [None] * T + [initial_particles]  # draw initial particles - put at index -1
    nu_weights = [None] * T  # these are nu weights
    mean_observation = [None] * T  # p(y_t|x_t)
    std_observation = [None] * T
    mean_state_prediction = [None] * T  # p(x_t|x_t-1)
    std_state_prediction = [None] * T
    mean_filtering = [None] * T  # p(x_t|x_t-1, y_t)
    std_filtering = [None] * T
    ancestor_indices = [None] * T
    loglikelihood = 0

    K = Q * C / (C * Q * C + R)
    state_proposal_stddev = np.sqrt((1 - K * C) * Q)
    obs_proposal_stddev = np.sqrt(C * Q * C + R)

    iterator = tqdm(range(T)) if verbose else range(T)
    for t in iterator:
        # WEIGHT
        # measurement
        fcn = step_x(particles[t-1], **step_kwargs)
        obs_mean = C * fcn
        measurement_proposal_dist = scipy.stats.norm(obs_mean, obs_proposal_stddev)  # p(y_t|x_t-1)

        # compute weights (nu)
        log_nu_weights_unnorm = measurement_proposal_dist.logpdf(y_data[t])
        nu_weights_unnorm = np.exp(log_nu_weights_unnorm - np.max(log_nu_weights_unnorm))
        nu_weights[t] = nu_weights_unnorm / np.sum(nu_weights_unnorm)

        # RESAMPLE
        a_indices = np.random.choice(range(N), p=nu_weights[t], replace=True, size=N)
        ancestor_indices[t] = a_indices

        # PROPAGATE
        # state
        fcn = step_x(particles[t-1][a_indices], **step_kwargs)
        state_mean = fcn + K * (y_data[t] - C * fcn)
        state_proposal_dist = scipy.stats.norm(state_mean, state_proposal_stddev)  # p(x_t|x_t-1^a_t,y_t)
        particles[t] = state_proposal_dist.rvs()

        # Store some statistics
        # marginal filtering mean and variance
        mean_filtering[t], std_filtering[t] = np.mean(particles[t]), np.std(particles[t])
        # prediction
        fcn = step_x(particles[t-1], **step_kwargs)  # this is done before resampling
        state_prediction_dist = scipy.stats.norm(fcn, np.sqrt(Q))  # p(x_t|x_t-1)
        mean_state_prediction[t] = np.mean(state_prediction_dist.mean())
        std_state_prediction[t] = np.mean(state_prediction_dist.std())
        # measurement
        measurement_dist = scipy.stats.norm(C * particles[t], np.sqrt(R))
        mean_observation[t] = np.mean(measurement_dist.mean())
        std_observation[t] = np.mean(measurement_dist.std())

        # likelihood
        log_obs = measurement_dist.logpdf(y_data[t])
        log_state_pred = state_prediction_dist.logpdf(particles[t])
        log_state_prop = state_proposal_dist.logpdf(particles[t])
        loglikelihood_term = log_obs + log_state_pred - log_state_prop - np.log(nu_weights[t][a_indices]) - np.log(N)
        loglikelihood += log_op_exp(loglikelihood_term, np.mean)

    nu_weights = np.array(nu_weights)
    particles = np.array(particles[:-1])  # remove initial state
    mean_filtering = np.array(mean_filtering)
    std_filtering = np.array(std_filtering)
    mean_state_prediction = np.array(mean_state_prediction)
    std_state_prediction = np.array(std_state_prediction)
    mean_observation = np.array(mean_observation)
    std_observation = np.array(std_observation)
    loglikelihood = np.array(loglikelihood)
    ancestor_indices = np.array(ancestor_indices)

    output = SimpleNamespace(
        nu_weights=nu_weights, particles=particles, mean_filtering=mean_filtering,
        std_filtering=std_filtering, mean_state_prediction=mean_state_prediction,
        std_state_prediction=std_state_prediction, mean_observation=mean_observation,
        std_observation=std_observation, loglikelihood=loglikelihood, ancestor_indices=ancestor_indices,
    )
    return output

In [ ]:
N = 5000
initial_particles = np.random.normal(0, 1, N)
output = fully_adapted_pf(initial_particles, step_x, C, Q, R, seed=None, theta=1)
output.loglikelihood

In [ ]:
np.mean(np.abs(x_data - output.mean_filtering)), np.mean(x_data - output.mean_filtering), np.var(x_data - output.mean_filtering)

In [ ]:
plt.plot(x_data - output.mean_filtering)

In [ ]:
plt.plot(x_data)
plt.plot(output.mean_filtering);

In [ ]:
plt.plot(output.std_filtering)

In [ ]:
def mh_correction(current, proposal, proposal_dist):
    proposal_relative = current - proposal + proposal_dist.mean()
    current_relative = proposal - current + proposal_dist.mean()
    proposal_prob = proposal_dist.logpdf(proposal_relative)
    current_prob = proposal_dist.logpdf(current_relative)
    return proposal_prob - current_prob


In [ ]:
def particle_metropolis_hastings(n_steps, initial_theta, random_walk_proposal, theta_prior, initial_particle_dist, n_particles, step_x, C, Q, R, verbose=0, seed=0):
    if seed is not None:
        np.random.seed(seed)

    current_theta = initial_theta
    initial_particles = initial_particle_dist.rvs(n_particles)
    output = fully_adapted_pf(initial_particles, step_x, C, Q, R, seed=None, verbose=verbose>1, theta=initial_theta)
    current_loglikelihood = output.loglikelihood

    thetas = []
    loglikelihoods = []
    iterator = tqdm(range(n_steps)) if verbose > 0 else range(n_steps)
    for m in iterator:
        proposed_theta = current_theta + random_walk_proposal.rvs() - random_walk_proposal.mean()

        initial_particles = initial_particle_dist.rvs(n_particles)
        output = fully_adapted_pf(initial_particles, step_x, C, Q, R, seed=None, verbose=verbose>1, theta=proposed_theta)
        proposed_loglikelihood = output.loglikelihood

        correction = mh_correction(current_theta, proposed_theta, random_walk_proposal)

        proposed_theta_logprob = theta_prior.logpdf(proposed_theta)
        current_theta_logprob = theta_prior.logpdf(current_theta)

        acceptance = proposed_theta_logprob - current_theta_logprob + proposed_loglikelihood - current_loglikelihood + correction
        event = np.log(np.random.uniform(0, 1))
        if acceptance > event:
            current_theta = proposed_theta
            current_loglikelihood = proposed_loglikelihood

        thetas.append(current_theta)
        loglikelihoods.append(current_loglikelihood)
        
    return thetas, loglikelihoods

In [ ]:
# Estimate parameters (infer posterior p(theta|y_data)) using Particle Metropolis Hastings
theta_prior = scipy.stats.norm(0, 1)  # p(theta) = N(0, 1)
theta_rw_proposal = scipy.stats.norm(0, 1)  # q(theta'|theta) = N(0, 0.1)
initial_particle_dist = scipy.stats.norm(0, 1)
initial_theta = 0.5  # theta_prior.rvs()
N = 500  # Number of APF particles
M = 500  # Number of PMH runs

In [ ]:
thetas, loglikelihoods = particle_metropolis_hastings(M, initial_theta, theta_rw_proposal, theta_prior, initial_particle_dist, N, step_x, C, Q, R, verbose=1, seed=0)

In [ ]:
len(thetas), len(loglikelihoods), len(np.unique(loglikelihoods)), round(len(np.unique(loglikelihoods)) / len(loglikelihoods), 2)

In [ ]:
thetas[np.argmax(loglikelihoods)]

In [ ]:
plt.hist(thetas, bins=50, density=True, label="Estimated posterior")
plt.plot([theta, theta], [0, 1], 'r', label='True value')
plt.xlabel("$\theta$")
plt.legend()

In [ ]:
plt.scatter(thetas, loglikelihoods);

The posterior distribution modes are biased compared to the true parameters, especially for a low number of timesteps.

Increasing the number of timesteps, however, will lead to a more accurate estimate of the posterior mode.

# Problem 2

We would like to use a Fully Adapted Particle Filter to implement the Conditional Particle Filter for the Particle Gibbs Sampler.

This is slightly more complicated than doing it with the Bootstrap Particle Filter due to the changed order of resample-propagate-weight and the extra set of weights.

Hence, we will use the Bootstrap Particle Filter to implement the Conditional Particle Filter for the Particle Gibbs Sampler.

In [ ]:
def backtrack_genealogy(list_index, list_sample):
    """Requires initial particle to be at list_sample[-1] and len(list_sample) = len(list_index) + 1"""
    aux_list_index = copy.deepcopy(list_index)
    genealogy = [list_sample[-2].reshape(1, -1)]

    T = len(list_index)
    for t in range(T - 1, -1, -1):  # [4, 3, 2, 1, 0]
        genealogy.insert(0, list_sample[t-1][aux_list_index[t]].reshape(1, -1))
        aux_list_index[t-1] = aux_list_index[t-1][aux_list_index[t]]

    genealogy = np.concatenate(genealogy, axis=0)  # (x_0, x_1, x_2, ..., x_T)
    return genealogy

In [ ]:
def weighted_mean_and_var(values, weights):
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)
    return (average, variance)

In [ ]:
def conditional_bootstrap_pf(initial_particles, reference_trajectory, step_x, C, Q, R, seed=0, verbose=True, **step_kwargs):
    """Conditional Bootstrap Particle Filter
    
    Reference trajectory should be given as [x_1, ..., x_T, x_0] i.e. with the initial state last.
    """
    np.random.seed(seed)
    N = len(initial_particles)
    if verbose:
        print(f"Running with {N} particles")
    initial_particles[N-1] = reference_trajectory[-1]  # deterministically set reference trajectory
    weights = [None] * T + [np.array([1/N] * N)]
    particles = [None] * T + [initial_particles]
    mean_filtering = [None] * T
    var_filtering = [None] * T
    ancestor_indices = [None] * T

    iterator = tqdm(range(T)) if verbose else range(T)
    for t in iterator:
        # RESAMPLE
        ancestor_indices[t] = np.random.choice(range(N), p=weights[t-1], replace=True, size=N-1)

        # PROPAGATE
        # state
        fcn = step_x(particles[t-1][ancestor_indices[t]], **step_kwargs)
        proposal_dist = scipy.stats.norm(fcn, np.sqrt(Q))
        particles[t] = proposal_dist.rvs()
        # deterministically set reference trajectory
        particles[t] = np.concatenate([particles[t], reference_trajectory[t]])
        ancestor_indices[t] = np.concatenate([ancestor_indices[t], [N-1]])
        # measurement
        fcn = C * particles[t]
        measurement_dist = scipy.stats.norm(fcn, np.sqrt(R))

        # WEIGHT
        log_weights_unnorm = measurement_dist.logpdf(y_data[t])
        weights_unnorm = np.exp(log_weights_unnorm - np.max(log_weights_unnorm))
        weights[t] = weights_unnorm / np.sum(weights_unnorm)

        mean_filtering[t], var_filtering[t] = weighted_mean_and_var(particles[t], weights[t])

    weights = np.array(weights)
    particles = np.array(particles)
    mean_filtering = np.array(mean_filtering)
    var_filtering = np.array(var_filtering)
    ancestor_indices = np.array(ancestor_indices)

    # Sample new reference trajectory
    genealogy = backtrack_genealogy(ancestor_indices, particles)
    j = np.random.choice(range(N), p=weights[T-1], replace=False, size=1)
    reference_trajectory = genealogy[:, j].reshape(-1)  # (T+1, N) -> (T+1,)
    reference_trajectory = np.concatenate([reference_trajectory[1:], reference_trajectory[:1]])  # put initial at end

    output = SimpleNamespace(
        weights=weights,
        particles=particles,
        mean_filtering=mean_filtering,
        var_filtering=var_filtering,
        ancestor_indices=ancestor_indices,
        genealogy=genealogy,
        reference_trajectory=reference_trajectory,
        loglikelihood=None,
    )
    return output

In [ ]:
N = 20
initial_particles = np.random.normal(0, 1, N)
initial_reference_trajectory = np.random.normal(5, 1, size=(T + 1, 1))
output = conditional_bootstrap_pf(initial_particles, initial_reference_trajectory, step_x, C, Q, R, seed=None, theta=1)
output.loglikelihood
output.reference_trajectory.shape

In [ ]:
def plot_genealogy(genealogy, particles, ancestor_indices, reference_trajectory=None, sampled_trajectory=None):
    fig, ax = plt.subplots(1, 1, figsize=(20, 10))

    T = len(genealogy) - 1

    ax.plot(list(range(T+1)), genealogy[:,:-1], marker='o', color='tab:red')
    ax.plot(list(range(T+1)), genealogy[:,-1:], marker='o', color='tab:red', label="Genealogy")

    for t in range(T):
        p = np.array([particles[t-1][ancestor_indices[t]], particles[t]])
        ax.plot([t, t+1], p, marker='o', color='silver', alpha=0.5)

    ax.plot([0, 0], [particles[0,:-1], particles[0,:-1]], marker='o', color='silver', alpha=1)
    ax.plot([0, 0], [particles[0,-1:], particles[0,-1:]], marker='o', color='silver', alpha=1, label="Particles")

    if reference_trajectory is not None:
        # put initial at front
        reference_trajectory_plot = np.concatenate([reference_trajectory[-1:], reference_trajectory[:-1]])
        ax.plot(reference_trajectory_plot, color="tab:blue", label="Reference trajectory")

    if sampled_trajectory is not None:
        # put initial at front
        sampled_trajectory_plot = np.concatenate([sampled_trajectory[-1:], sampled_trajectory[:-1]])
        ax.plot(sampled_trajectory_plot, color="tab:green", label="Sampled trajectory")

    ax.legend()
    return fig, ax

In [ ]:
genealogy = output.genealogy
particles = output.particles
ancestor_indices = output.ancestor_indices

plot_genealogy(genealogy, particles, ancestor_indices, initial_reference_trajectory, output.reference_trajectory)

# Problem 3

# Problem 4

# Problem 5

# Problem 6